In [ ]:
import json

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd
from ghapi.all import GhApi, pages

In [ ]:
with open(".secrets.json") as read_file:
    api = GhApi(token=json.load(read_file)["api_token"])

In [ ]:
api

In [ ]:
# repo_list = ["matthewfeickert/student-developer-reporting-demo"]
repo_list = ["scikit-hep/pyhf", "scikit-hep/hist"]
developer_username = "matthewfeickert"

In [ ]:
def get_data(repositories, username):
    # This should be a dataframe
    data = []
    for repository in repositories:
        owner, repo = repository.split("/")
        per_page = 30

        _ = api.issues.list_for_repo(owner, repo, per_page=per_page, creator=username)
        n_pages = api.last_page()
        n_pages = n_pages if n_pages > 0 else 1

        all_pages = pages(
            api.issues.list_for_repo,
            n_pages,
            owner,
            repo,
            per_page=per_page,
            creator=username,
            headers={"Accept": "application/vnd.github.v3.issues+json"},
        )
        data.extend(
            {"repo": repository, "date": item.created_at, "count": 1}
            for page in all_pages
            for item in page
        )

    return pd.DataFrame(data)

In [ ]:
data_df = get_data(repo_list, developer_username)
data_df

In [ ]:
df = data_df.copy()

df = df[df.repo == "scikit-hep/pyhf"]
# df = df[df.repo == "scikit-hep/hist"]

df["date"] = pd.to_datetime(df.date)
df = df.resample("W-Mon", on="date").sum(numeric_only=True).cumsum()

fig, ax = plt.subplots()

time_fmt = mdates.AutoDateLocator()
ax.xaxis.set_major_locator(time_fmt)

ax.plot(df.index, df["count"], label="x")
ax.legend(loc="best", frameon=False)

ax.set_xlabel("Date", size=14)
ax.set_ylabel(f"Number of GitHub Issues opened", size=14)

fig.tight_layout()

# df["date"] = pd.to_datetime(df["date"])
# df = df.resample("W-Mon", on="date").sum().cumsum()

In [ ]:
_repo = repo_list[0]

df = data_df.copy()

df["date"] = pd.to_datetime(df["date"])
df = df.resample("W-Mon", on="date").sum(numeric_only=True).cumsum()
df["repo"] = _repo

fig, ax = plt.subplots()

time_fmt = mdates.AutoDateLocator()
ax.xaxis.set_major_locator(time_fmt)

ax.plot(df.index, df["count"], label=_repo)
ax.legend(loc="best", frameon=False)

ax.set_xlabel("Date", size=14)
ax.set_ylabel(f"Number of GitHub Issues opened", size=14)

fig.tight_layout()

file_types = ["png", "pdf", "svg"]
for extension in file_types:
    fig.savefig(
        f"ghapi_{_repo.split('/')[1]}_issue_history.{extension}", facecolor="white"
    )

---

In [ ]:
activity = api.activity.list_public_events_for_user(developer_username, 30, 1)[4]

In [ ]:
if activity.repo.name in repo_list:
    print("yay")

    if "issue" in activity.payload:
        print(activity.payload.keys())
        if activity.payload.action == "opened":
            print(
                f"{developer_username} opened an issue on {activity.repo.name} as {activity.created_at}"
            )

# activity